# SQL 3 Exercises

---

## Question 1

Write SQL statements to create the tables `employee` & `department` with the following constraints and attributes respectively. You can test your statements by running the SQL statement in your MariaDB.

| Table | Attributes |
|:---|:---|
| employee | id (PK, char(5))<br>name (not null, varchar)<br>deptID (fk dept(dept_id), char(5))<br>salary (int, 22000 < salary<  50000)<br>insurance_opted (char, default value 'Y') |
| department | dept_id (Unique, not null, char(5))<br>name (unique, varchar) |


**Answer**
```sql
CREATE TABLE department(dept_id CHAR(5) NOT NULL UNIQUE,
name VARCHAR(20) UNIQUE);


CREATE TABLE employee (ID CHAR(5),
name VARCHAR(20) NOT NULL,
deptid CHAR(5),
salary INT,
insurance_opted CHAR(1) DEFAULT 'Y',
PRIMARY KEY(ID), 
FOREIGN KEY(deptid) REFERENCES department(dept_id),
CHECK(salary > 22000 AND salary < 50000));
```

## Question 2
In this question we are going to investigate how we can call a stored procedure from a trigger

1. Write the SQL statement to create a table called `accounts` with the following constraints in MariaDB. Then insert a few records into the table.

| Table | Attributes |
|:---|:---|
| accounts | acct_id (PK, auto increment)<br>name (not null, varchar)<br>amount (decimal(10,2), not null, amount>=0) |

**Answer**
```sql
CREATE TABLE accounts (
    acct_id INT AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    amount DECIMAL(10 , 2 ) NOT NULL ,
    PRIMARY KEY (acct_id),
    CHECK(amount >= 0) 
);
```

2. Write a stored procedure that withdraws from an account. Remember to use the `DELIMITER` keyword.<br>
 **Pseudocode:**
 ```
 if withrawal amount is <= 0
     throw SQL exception with message
 
 update the accounts table
 ```

**Answer**
```sql
DELIMITER //

CREATE PROCEDURE Withdraw(IN fromAccountId INT, IN withdrawAmount DEC(10,2))
BEGIN
    IF withdrawAmount <= 0 THEN
        SIGNAL SQLSTATE '45000' 
            SET MESSAGE_TEXT = 'The withdrawal amount must be greater than zero';
    END IF;
    
    UPDATE accounts 
    SET amount = amount - withdrawAmount
    WHERE acct_id = fromAccountId;
END //

DELIMITER ;
```

3. Write a stored procedure that checks the withdrawal from an account. Remember to use the DELIMITER keyword.<br>
 **Logic:**
 ```
 declare some variables
 using a select statement get the current balance out from the account table and store it
 
 store the new balance amount after the withdrawal
 if new balance amount < 2 /* minimum balance to maintain */
     throw SQL exception with message (you may need to refer to mariadb docs to see the string functions)
 ```

**Answer**

```sql
DELIMITER //

CREATE PROCEDURE CheckWithdrawal(IN fromAccountId INT, IN withdrawAmount DEC(10,2))
BEGIN
    DECLARE balance DEC(10,2);
    DECLARE newBal DEC(10,2);
    DECLARE message VARCHAR(255);

    -- get current balance of the account
    SELECT amount INTO balance FROM accounts
    WHERE acct_id = fromAccountId;
    
    -- check balance
    SET newBal = balance - withdrawAmount;

    IF newBal < 2 THEN
        SET message = CONCAT('Insufficient amount, your balance is ', balance);
        SIGNAL SQLSTATE '45000' 
            SET MESSAGE_TEXT = message;
    END IF;
END //

DELIMITER ;
```

4. Test your stored procedure from step 3 before moving on to step 5. Refer to the lecture notes on how to invoke a stored procedure.

5. Write a trigger that calls the stored procedure from step 3. This trigger is to fire **before the update** to the `accounts` table.<br>
 **Hint**: Invoke the stored procedure from within the `BEGIN...END` block in the trigger.

**Answer**
```sql
DELIMITER //

CREATE TRIGGER before_accounts_update
BEFORE UPDATE
ON accounts FOR EACH ROW
BEGIN
    CALL CheckWithdrawal (
        OLD.acct_id, 
        OLD.amount - NEW.amount
    );
END //

DELIMITER ;
```

6. Test your transaction by invoking the stored procedure in step 2. Use `SELECT` statements to check your results. You can reset the account amounts using the regular `UPDATE` statements.

**Answer**
```sql
/* withdrawing from account 1 */
CALL withdraw(1,25);

SELECT * FROM accounts;
```

## Question 3

In this question we are going to investigate how we can chain triggers on the same table.

From the lecture, the general syntax for a Trigger is:
```sql
CREATE TRIGGER trigger_name
{BEFORE | AFTER} {INSERT | UPDATE | DELETE }
ON table_name FOR EACH ROW
[{ FOLLOWS | PRECEDES } other_trigger_name ]
trigger_body;
```

the `FOLLOWS` & `PRECEDES` specifies whether the other trigger should be invoked before or after an existing trigger.

To keep this investigation small, we will import the tables from the file `chaining_triggers.sql` (use the same instructions for importing databases into MariaDB as in SQL 2 Exercise). Once imported you should have the following tables and some data included in the `products` table.

```
products(productCode PK, productName, productLine, productScale, productVendor, productDescription, quantityInStock, buyPrice, MSRP)
priceLogs(id PK, productCode FK, price, updated_at)
userChangeLogs(id PK, productCode FK, updatedAt, updatedBy)
```
You are encouraged to open the file to see how it is structured. 

**Objective**<br>
Log the changes made to the the price of a product (column `MSRP` in the `products` table) when it's updated.

1. Create a Trigger that inserts a record to the `pricelogs` table **before the update** to the `products` table, `MSRP` column. The column `updated_at` can be omitted because the constraint attached to that attribute ensures that it will get filled with a value upon creation.  

**Answer**
```sql
DELIMITER //

CREATE TRIGGER before_products_update 
   BEFORE UPDATE ON products 
   FOR EACH ROW 
BEGIN
     IF OLD.msrp <> NEW.msrp THEN
         INSERT INTO PriceLogs(productCode,price)
         VALUES(old.productCode,old.msrp);
     END IF;
END //

DELIMITER ;
```

2. Test your trigger by updating the `MSRP` price of any record in the `products` table. You should see a record get created in the `pricelogs` table.

3. Create another Trigger that inserts a record to the `userChangeLogs` tables **before the update** to the `products` table, `MSRP` column. This trigger is to be activated **after** the trigger created in step 1. Again the `updatedAt` column can be omitted because the constraint attached to that attribute ensures that it will get filled with a value upon creation. However, to get the current user id, use the `USER()` function.

**Answer**
```sql
DELIMITER //

CREATE TRIGGER before_products_update_log_user
   BEFORE UPDATE ON products 
   FOR EACH ROW 
   FOLLOWS before_products_update
BEGIN
    IF OLD.msrp <> NEW.msrp THEN
    INSERT INTO UserChangeLogs(productCode, updatedBy)
        VALUES (OLD.productCode, USER());
    END IF;
END //

DELIMITER ;
```

4. Test the tigger created in step 3 by updating the `MSRP` price of any record in the `products` table. You should now see a record created in both the `pricelogs` and `userChangeLogs` table.

**Bonus**<br>
Sometimes we need to check the trigger order of the created triggers. This information is stored in the `information_schema` database, `triggers` table. We can access it via the following SQL

```sql
SELECT 
    trigger_name, 
    action_order
FROM
    information_schema.triggers
WHERE
    trigger_schema = <database_name>
ORDER BY 
    event_object_table , 
    action_timing , 
    event_manipulation;
```